In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import networkx as nx

In [2]:
np.random.seed(0)

In [3]:
embeddings = {}
with open('All-train.embeddings') as f:
    node_n, dim = map(int, f.readline().split())
    print(node_n, dim)
    for line in f:
        #print(line.split())
        tmp = list(map(float, line.split()))
        #print(tmp)
        id = int(tmp[0])
        em = tmp[1:]
        embeddings[id] = em

29402 128


In [4]:
G = nx.read_edgelist('../All-train.txt', nodetype=int, create_using=nx.DiGraph()) 

In [5]:
negative = []
for node in G:
    for first in G[node]:
        for second in G[first]:
            negative.append((node, second))
negative = np.array(negative)
#print(len(negative))

In [6]:
import random
sample = np.random.choice(len(negative)+1, G.number_of_edges())

In [7]:
print(negative[sample].shape)

(285789, 2)


In [8]:
positive = []
with open('../All-train.txt') as f:
    for line in f:
        positive.append(np.array(list(map(int, line.split()))))
positive = np.array(positive)
print(positive.shape)

(285789, 2)


In [9]:
X = np.vstack((positive, negative[sample]))
Y = [1] * len(positive) + [0] * len(negative[sample])

In [10]:
data = []
for id1, id2 in X:
    data.append(np.append(embeddings[id1], embeddings[id2]))
data = np.array(data)
print(data.shape)

(571578, 256)


In [32]:
from sklearn import svm
g_svm = svm.LinearSVC(C=1e10, dual=False, verbose=True)

In [33]:
g_svm.fit(data, Y)

[LibLinear]

LinearSVC(C=10000000000.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=True)

In [34]:
pred = []
with open('../t1-test.txt') as f:
    for line in f:
        id1, id2 = map(int, line.split())
        try:
            testdata = np.append(embeddings[id1], embeddings[id2])
            tmp = g_svm.predict([testdata])
            pred.append(tmp[0])
        except:
            pred.append(1)
pred = np.array(pred)

In [35]:
np.savetxt('pred.txt', pred.reshape(-1, 1), fmt='%d')

In [36]:
!python3 ../pred-txt-to-csv.py pred.txt